In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Define tools

In [ ]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_result=2)
search_results = search.invoke("What is weather in San Francisco?")
print(search_results)


{'query': 'What is weather in San Francisco?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752692968, 'localtime': '2025-07-16 12:09'}, 'current': {'last_updated_epoch': 1752692400, 'last_updated': '2025-07-16 12:00', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 12.8, 'wind_kph': 20.5, 'wind_degree': 243, 'wind_dir': 'WSW', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 68, 'cloud': 50, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'windchill_c': 13.5, 'windchill_f': 56.3, 'heatindex_c': 14.5, 'heatindex_f': 58.0, 'dewpoint_c'

In [3]:
tools = [search]

### Using Lnaguage Models

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="gpt-4o-mini", temperature=0.1, model_provider="openai")

In [5]:
query = "Hi"
response = model.invoke([{"role": "user", "content": query }])
response.content

'Hello! How can I assist you today?'

In [6]:
model_with_tools = model.bind_tools(tools)

In [ ]:
query = "Hi"
response = model_with_tools.invoke([{"role": "user", "content": query }])
response.content

'Hello! How can I assist you today?'

In [8]:
response.tool_calls

[]

In [14]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in San Francisco', 'topic': 'general'}, 'id': 'call_s2FV7VKrAHqeiM25IlmVxYxV', 'type': 'tool_call'}]


### Create Agent

In [15]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### Run the agent

In [16]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]: 
   message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


In [17]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]: 
   message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_VeiMyB0FXGzTD5Wi6rM4VCwm)
 Call ID: call_VeiMyB0FXGzTD5Wi6rM4VCwm
  Args:
    query: weather in San Francisco
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752695046, 'localtime': '2025-07-16 12:44'}, 'current': {'last_updated_epoch': 1752694200, 'last_updated': '2025-07-16 12:30', 'temp_c': 17.8, 'temp_f': 64.0, 

### Streaming Messages

In [28]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
   step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_2Iy5vg2RtZZYhv4PcZUGsnUP)
 Call ID: call_2Iy5vg2RtZZYhv4PcZUGsnUP
  Args:
    query: weather in San Francisco
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752695046, 'localtime': '2025-07-16 12:44'}, 'current': {'last_updated_epoch': 1752694200, 'last_updated': '2025-07-16 12:30', 'temp_c': 17.8, 'temp_f': 64.0, 

### Streaming tokens

In [29]:
for step, metadata in agent_executor.stream({"messages": [input_message]}, stream_mode="messages"): 
   if metadata["langgraph_node"] == "agent" and (text := step.text()): 
      print(text, end="|")

The| current| weather| in| San| Francisco| is| as| follows|:

|-| **|Temperature|**|:| |17|.|8|°C| (|64|.|0|°F|)
|-| **|Condition|**|:| Part|ly| cloudy|
|-| **|Wind|**|:| |12|.|8| mph| (|20|.|5| k|ph|)| from| the| W|SW|
|-| **|Humidity|**|:| |68|%
|-| **|Visibility|**|:| |16| km| (|9| miles|)
|-| **|Pressure|**|:| |101|5| mb| (|29|.|98| in|)

|For| more| detailed| information|,| you| can| check| the| following| sources|:
|-| [|Weather| API|](|https|://|www|.weather|api|.com|/)
|-| [|Acc|u|Weather|](|https|://|www|.acc|uwe|ather|.com|/en|/us|/s|an|-fr|anc|isco|/|941|03|/j|uly|-weather|/|347|629|)|

### Adding in memory

In [30]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [32]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [38]:
input_message = {"role": "user", "content": "Hi I'm Pon"}

for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
   step["messages"][-1].pretty_print() 

================================ Human Message =================================

Hi I'm Pon
================================== Ai Message ==================================

Hello again, Pon! If there's anything you'd like to discuss or any questions you have, feel free to let me know!


In [39]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Pon!
